<a href="https://colab.research.google.com/github/schwarzmarcel/MasterThesis_MSchwarz/blob/main/AI_Method_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import necessary libraries


In [ ]:
import pandas as pd
import numpy as np
import json
import codecs
import re

# Build AI methods list


*   load the text file with the AI methods
*   build a python list containing the terms and synonyms



In [ ]:
ai_method_file = "ai_method_list.txt"

In [ ]:
def build_method_list(ai_method_file):
  with codecs.open(ai_method_file, "r", encoding="utf8") as f_in:
    ai_methods = f_in.read().split("\n")
    method_list = list()

    for term in ai_methods:
      values = term.split(" | ")
      terms = list()
      terms.append(values[0].replace("-", " ").lower())
      values[1] = values[1].strip("\r")
      if values[1] != "":
        for synonym in values[1].split(","):
          terms.append(synonym)
      #print(terms)
      method_list.append(terms)
      
  f_in.close()

  return method_list

# Define methods to search terms within a text


In [ ]:
def search_term(text, term):
  for synonym in term:
     if text.__contains__(synonym):
       return True
  return False

In [ ]:
def search_methods(text, method_list):
  text = text.lower().replace(",", "").replace(".", "").replace("-", " ")
  found_terms = list()
  for term in method_list:
    if search_term(text, term):
      found_terms.append(term)

  return(found_terms)

# Measure percentage of AI terms found within the summary

To do this, you simply need a source text and a target text to measure the overlap of methods. The following code snippet can be used with a dataframe that contains the abstract texts, gold summaries and predicted summaries. First the source text (either abstract or gold summary, depending on the metric), is searched for terms. Then, all the terms which were found are searched in the generated summary to calculate how many terms where included.

In [ ]:
method_list = build_method_list(ai_method_file)
all_terms = 0
all_hits = 0

for i in range(df.PMID.size):

  sentences = df.iloc[i, 1]
  abstract = " ".join(sentences)

  gold_summary = list()
  for n, idx in enumerate(df.iloc[i, 3]):
    if idx == 1:
      gold_summary.append(sentences[n])
  gold_summary = " ".join(gold_summary)

  summary = list()
  for n, idx in enumerate(df.iloc[i, 8]):
    if idx == 1:
      summary.append(sentences[n])
  summary = " ".join(summary)

  found_terms = search_methods(abstract, method_list) # search_methods(gold_summary, method_list)
  hits = search_methods(summary, found_terms)
  all_terms += len(found_terms)
  all_hits += len(hits)

print(all_terms)
print(all_hits / all_terms)